In [1]:
import spacy
import pandas as pd
import os

In [2]:
from lxml import etree
from datetime import datetime

In [3]:
def month_dates(start, end):
    f = lambda date: date.month + 12 * date.year

    res = []
    for tot_m in range(f(start)-1, f(end)):
        y, m = divmod(tot_m, 12)
        res.append(str(y) + '/' + '%02d' % (m+1))
    
    return res

In [4]:
def get_date(article):
    """
    This method returns the date of the article
    """
    str_date = article.find('entity').find('meta').find('issue_date').text
    return datetime.strptime(str_date, '%d/%m/%Y')

In [5]:
def get_articles_in_file(file, start_date, end_date):
    articles = []  
    for article in file.iter('article'):
        if article.find('entity') is not None:
            a = ''
            date = get_date(article)
            if start_date <= date <= end_date:
                for entity in article.iter('entity'):
                    a += entity.findtext('full_text') + ' '
                articles.append(date.strftime('%d/%m/%Y') + ' ' + a)
    return articles

In [6]:
def get_articles(path, start_date, end_date):
    articles = []
    for m_date in month_dates(start_date, end_date):
        try:
            file = etree.parse(path + m_date + '.xml')
            articles.append(get_articles_in_file(file, start_date, end_date))
        except (FileNotFoundError, IOError):
            pass
    return [a for file in articles for a in file]  

In [28]:
def get_specific_entity(elementTree, box_id):
    value = elementTree.findall('box')
    return value

file = etree.parse('/home/mbanga/Desktop/JDG/1990/01.xml')
print(file.findall('entity'))

[]


In [7]:
path = '/home/mbanga/Desktop/JDG/'
start_date =  datetime(1990, 1, 1)
end_date = datetime(1990, 1, 31)

In [8]:
articles = get_articles(path, start_date, end_date)

In [9]:
len(articles)

3434

In [10]:
import fr_core_news_sm
import enchant

In [11]:
nlp = fr_core_news_sm.load()

In [12]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuaiton or whitespace
    """
    
    return token.is_punct or token.is_space

In [13]:
def is_french(word):
    """
    helper function to eliminate tokens that
    are not french words.
    """
    d = enchant.Dict('fr_FR')
    return d.check(word)

In [14]:
def lemmatized_corpus(corpus):
    """
    generator function to use spaCy to parse articles,
    lemmatize the text, and yield sentences
    """
    j = 0
    i = 0
    for parsed_article in nlp.pipe(corpus, 
                                   batch_size=100, n_threads=5):
        # save the date
        date = parsed_article[0].text

        yield (date, ' '.join([token.lemma_ for token in parsed_article
                             if not punct_space(token) and is_french(token.text)
                                and not token.is_stop and not token.is_digit
                                and not token.like_num]))

In [24]:
if 0 == 1:
    %%time
    # Time consuming !!
    lemmatized_corpus = [(date, lemmas) for date, lemmas in lemmatized_corpus(articles[:30])]

    # retrieve dates
    dates = [pair[0] for pair in lemmatized_corpus]

    # retrieve articles
    corpus = [pair[1] for pair in lemmatized_corpus]

In [ ]:
with open(os.path.join(project_path, 'lemmatized articles january 1990.txt'), 'w') as file:
    for article in lemmatized_articles:
        file.write(article + u'\n')

In [16]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora import Dictionary, MmCorpus

import pyLDAvis
import pyLDAvis.gensim
import warnings
#import cPickle as pickle

In [ ]:
# learn the dictionnary by iterating over all of the articles
dico = Dictionary([article.split() for article in corpus])

# filter tokens that are very rare or too common from
# the dictionary 
dico.filter_extremes(no_below=1, no_above=0.2)

# reassign integer lda
dico.compactify()

In [ ]:
def bow_generator(corpus):
    """
    generator function to read articles from a file
    and yield a bag-of-words representation
    """
    for article in corpus:
        yield dico.doc2bow(article.split())

In [ ]:
# generate bag-of-word representations for
# all reviews and save them as a matrix
project_path = '/home/mbanga/Epfl/AppliedDataAnalysis/ADA2017_GroupWork/Project/'
MmCorpus.serialize(os.path.join(project_path, 'corpus.mm'),
                                bow_generator(corpus))

bow_corpus = MmCorpus(os.path.join(project_path, 'corpus.mm'))

In [ ]:
lda_model_filepath = os.path.join(project_path, 'lda_model_all')

In [ ]:
if 1 == 1:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(bow_corpus,
                           num_topics=15,
                           id2word=dico,
                           workers=3)
        
        lda.save(lda_model_filepath)

#load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

In [ ]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
    
    print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
    
    for term, frequency in lda.show_topic(topic_number, topn=10):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [ ]:
explore_topic(topic_number=14)

In [ ]:
lda[bow_corpus[0]]

In [ ]:
corpus[1101]

In [23]:
text = ['01/01/1990 La votation populaire du 12 janvier 1993']
text = ["Or asp ar. I .. Ig e r m  a.', ~ ,,','.', ",'., pi','ns, K'ASPAR VILLIGER est mal pris. Arrivé en,, 1989 à la tête du DMF, il hérite d'Arnold Koller h 3 projet d'achat du F-18 américain : 1989, c'est aussi en Suisse l'année du projet controversé « Diamant .., de la votation sur l'initiative'pour une Suisse sans armée (refusée par deux tiers des votants), et sur la scène internationale d'importants changl : lments à l'Est .:',. Dans le dossier F-18, Kaspar Villiger veut néanmoins aller de l'avant. Malgré les multiples attaques contre le projet (3 milliards de francs pour 34 appareils), il refuse l'idée d'une .. pause réüexlon ». En revanche, il entre en matière sur le mode d'acquisition en envisageant deux étapes (24 +'10). Malheureusement. au lieu de simplifier :, le débat, ce marchandage à la baisse t'opaclfle. --... ; .-- Gare aux pis aller.. • Pourquoi ? Eh bien, il faut tout d'abord rappeler que lorsque les experts du DMF ont décidé d'opter pour le F-18, à la suite de tests, très poussés, ils ont à juste titre souligné qu'une commande de 34 appareils représentait un minimum. En fait, il en faudrait une quarantaine. Pensons aux 108 F-16 de la Belgique ou aux 240 Viggen de la Suède. Et', neque parlons pas de la Corée du Sud, qui selon la revue . Aviati ? n International » de [anvler, vient de : ~ éclder d acheter 120 F- ~ 8, pour envlron ~; ml ! hards de dollars (4, 5 milliards de francs suisses) .. ", II faut être clair. Soit la situation stratégique internationale exige que la Suisse dispose de 34 F-18 et il n'y a aucune raison de revenir en arrière, soit elle a évolué de telle manière qu'elle ne l'exige plus. Pour pouvoir se prononcer valablement sur ce dernier point, il faut toutetols procéder à une nouvelle évaluation de la menace. Sans unè justification militaire fondée, la solution .. 24 + 10 .. passe en effet pour un pis-aller et donne l'impression. çe'c ¢ der du terrain sous la pression politi-. Si le Gouvernement veut acheter des F-18, il doit convaincre le peuple que c'est un besoin et une bonne solution face aux exigences de laguerre moderne. Il faut commencer par là. " José Bessard"]

for parsed_article in nlp.pipe(text, batch_size=100, n_threads=5):
     # save the date
        print(parsed_article[0].text)

        print(' '.join([token.lemma_ for token in parsed_article
                             if not punct_space(token) and is_french(token.text)
                                and not token.is_stop and not token.is_digit
                                and not token.like_num]))


01/01/1990
la votation populaire janvier


<generator object lemmatized_corpus at 0x7fb71ada10a0>